<a href="https://colab.research.google.com/github/gyanoday06/sentiment_analysis_ds/blob/main/Kaggle_Data_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
angry = pd.read_excel('kaggle/Emotion_angry.xlsx')
sad = pd.read_excel('kaggle/Emotion_sad.xlsx')
happy = pd.read_excel('kaggle/Emotion_happy.xlsx')

In [ ]:
frames = [sad, happy, angry]
df = pd.concat(frames)
df = df.drop_duplicates(subset='content', keep="first")
df.head()

,content,sentiment
0,"Never hurt people who love you a lot, because ...",sad
1,Don‚Äôt expect me to tell you what you did wro...,sad
2,I preferred walking away than fighting for you...,sad
3,"Moving forward in life isn‚Äôt the hard part, ...",sad
4,"Never cry for anyone in your life, because tho...",sad


In [ ]:
df['sentiment'].value_counts()

happy    702
angry    494
sad      390
Name: sentiment, dtype: int64

In [ ]:
#1. Naive Bayes Classifier (using scikit-learn):
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['sentiment'], test_size=0.2, random_state=42)

# Convert text data to numerical features using CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vectorized, y_train)

# Predictions
y_pred = nb_classifier.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Naive Bayes Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Naive Bayes Accuracy: 0.720125786163522
Classification Report:
               precision    recall  f1-score   support

       angry       0.69      0.76      0.72        93
       happy       0.79      0.69      0.73       141
         sad       0.66      0.73      0.69        84

    accuracy                           0.72       318
   macro avg       0.71      0.73      0.72       318
weighted avg       0.73      0.72      0.72       318



In [ ]:
#LTSM Model:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Convert categorical labels to numeric format
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['sentiment'])
num_classes = len(label_encoder.classes_)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2, random_state=42)

# Tokenize and pad the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq)
X_test_padded = pad_sequences(X_test_seq, maxlen=X_train_padded.shape[1])

In [ ]:
# Build a simple LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=X_train_padded.shape[1]))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model
y_pred_prob = model.predict(X_test_padded)
y_pred = y_pred_prob.argmax(axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Epoch 1/5
16/16 [==============================] - 98s 6s/step - loss: 1.0783 - accuracy: 0.4379 - val_loss: 1.0605 - val_accuracy: 0.4488
Epoch 2/5
16/16 [==============================] - 94s 6s/step - loss: 1.0307 - accuracy: 0.4615 - val_loss: 1.0031 - val_accuracy: 0.4646
Epoch 3/5
16/16 [==============================] - 99s 6s/step - loss: 0.8544 - accuracy: 0.6065 - val_loss: 0.8919 - val_accuracy: 0.6142
Epoch 4/5
16/16 [==============================] - 97s 6s/step - loss: 0.6282 - accuracy: 0.7949 - val_loss: 0.8173 - val_accuracy: 0.6378
Epoch 5/5
10/10 [==============================] - 11s 988ms/step
Accuracy: 0.6761006289308176
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.70      0.67        93
           1       0.75      0.75      0.75       141
           2       0.58      0.52      0.55        84

    accuracy                           0.68       318
   macro avg       0.66      0.66      0.66       318

In [ ]:
#CNN

In [ ]:
#CNN

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, classification_report

# Tokenize and pad the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq)
X_test_padded = pad_sequences(X_test_seq, maxlen=X_train_padded.shape[1])

# Build a simple CNN model
model_cnn = Sequential()
model_cnn.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=X_train_padded.shape[1]))
model_cnn.add(Conv1D(128, 5, activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dense(num_classes, activation='softmax'))

# Compile the model
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_cnn.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model
y_pred_prob_cnn = model_cnn.predict(X_test_padded)
y_pred_cnn = y_pred_prob_cnn.argmax(axis=1)

accuracy_cnn = accuracy_score(y_test, y_pred_cnn)
print(f"Accuracy (CNN): {accuracy_cnn}")
print("Classification Report (CNN):\n", classification_report(y_test, y_pred_cnn))

Epoch 1/5
16/16 [==============================] - 28s 2s/step - loss: 1.0663 - accuracy: 0.4310 - val_loss: 1.0425 - val_accuracy: 0.4488
Epoch 2/5
16/16 [==============================] - 27s 2s/step - loss: 0.9921 - accuracy: 0.4566 - val_loss: 0.9779 - val_accuracy: 0.4961
Epoch 3/5
16/16 [==============================] - 30s 2s/step - loss: 0.7965 - accuracy: 0.7258 - val_loss: 0.8261 - val_accuracy: 0.6457
Epoch 4/5
16/16 [==============================] - 27s 2s/step - loss: 0.4703 - accuracy: 0.9260 - val_loss: 0.7188 - val_accuracy: 0.6850
Epoch 5/5
10/10 [==============================] - 3s 345ms/step
Accuracy (CNN): 0.7327044025157232
Classification Report (CNN):
               precision    recall  f1-score   support

           0       0.78      0.67      0.72        93
           1       0.73      0.87      0.79       141
           2       0.70      0.57      0.63        84

    accuracy                           0.73       318
   macro avg       0.73      0.70      0.7